In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import datasets, models, transforms
import numpy as np
import os
from pathlib import Path
import torch.nn as nn
import torch
import numpy as np

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/lfw-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/lfw-dataset


In [3]:
folders = [name for name in os.listdir('/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled')]

In [4]:
folders = sorted(folders)
path = '/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
images_num = 0
for name in folders:
  files = os.listdir(f'/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/{name}')
  image_files = [f for f in files if f.lower().endswith('.jpg')]
  images_num += len(image_files)

print("Number of images: ", images_num)

Number of images:  13233


In [5]:
from itertools import combinations
import random
dataset_path = "/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled"

# Собираем список людей и их изображений
people_images = {person: os.listdir(os.path.join(dataset_path, person))
                 for person in os.listdir(dataset_path)
                 if os.path.isdir(os.path.join(dataset_path, person))}

positive_pairs = []
negative_pairs = []

# Создаем пары
for person, images in people_images.items():
    if len(images) > 1:
        positive_pairs += [(i,1) for i in combinations(images, 2)] # Все возможные пары для одного человека
# пары без повторений и учета порядка - уникальные
# Создаем негативные пары
people = list(people_images.keys())
for _ in range(len(positive_pairs)):  # Делаем столько же негативных примеров, сколько позитивных
    p1, p2 = random.sample(people, 2)
    img1 = random.choice(people_images[p1])
    img2 = random.choice(people_images[p2])
    negative_pairs.append(((img1, img2),-1))

print(f"✔ Создано {len(positive_pairs)} позитивных пар и {len(negative_pairs)} негативных пар.")
pairs = positive_pairs + negative_pairs

✔ Создано 242257 позитивных пар и 242257 негативных пар.


In [87]:
pairs = pairs[:100000]

In [20]:
n = Image.open('/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/Abid_Hamid_Mahmud_Al-Tikriti/Abid_Hamid_Mahmud_Al-Tikriti_0001.jpg')
print(n.size)

(250, 250)


In [6]:
class FaceEshkereDataset(Dataset):
  def __init__(self, pairs, mode):
    self.pairs = pairs
    self.mode = mode
    self.transform_train = transforms.Compose([
      transforms.Resize((112, 112)),
      transforms.RandomHorizontalFlip(p=0.2),  # Горизонтальный флип
      #transforms.ColorJitter(brightness=0.5, contrast=0.5),  # Изменение яркости и контраста
      transforms.ToTensor()
      ])
    self.transform_test = transforms.Compose([
      transforms.Resize((125, 125)),
      transforms.ToTensor()
      ])
  def __len__(self):
    return len(self.pairs)

  def __getitem__(self, index):
    path_to_image0 = Path(f"/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/{self.pairs[index][0][0][:-9]}/{self.pairs[index][0][0]}") #надо бы поменять конечно, но работает
    path_to_image1 = Path(f"/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/{self.pairs[index][0][1][:-9]}/{self.pairs[index][0][1]}")

    label = self.pairs[index][1]

    if self.mode == 'train':
      image0 = self.transform_train(Image.open(path_to_image0))
      image1 = self.transform_train(Image.open(path_to_image1))
    if self.mode == 'test':
      image0 = self.transform_test(Image.open(path_to_image0))
      image1 = self.transform_test(Image.open(path_to_image1))

    return image0, image1, label


In [7]:
from sklearn.model_selection import train_test_split

train_pairs, test_pairs = train_test_split(pairs, test_size=0.3, random_state=42)



In [9]:
train_dataset = FaceEshkereDataset(train_pairs, 'train')
test_dataset = FaceEshkereDataset(test_pairs, 'test')
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

In [107]:
len(train_dataset)

339159

In [24]:
from torchvision.models import resnet18
from tqdm import tqdm, tqdm_notebook

model = resnet18(pretrained=True)

for param in model.parameters():
  param.requires_grad = True
model.fc = nn.Linear(512,128, bias=True)

In [25]:
import torch
from tqdm import tqdm
from torch.nn import DataParallel

def train(model, optimizer, criterion, epochs, train_dataloader, test_dataloader):
    summary_loss_train = []
    summary_loss_test = []
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs") # штука для 2 и более гпу
        model = DataParallel(model)
    
    for epoch in tqdm(range(epochs), desc="Epochs", ncols=100):
        train_loss = 0
        train_norm_variable = 0
        model.train()
        for image0, image1, label in train_dataloader:
            image0 = image0.to(device)
            image1 = image1.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            
            embed0 = model(image0)
            embed1 = model(image1)
            
            loss = criterion(embed0, embed1, label)
            summary_loss_train.append(loss.item())
            train_loss += loss.item() * image0.size(0)
            train_norm_variable += image0.size(0)
            loss.backward()
            optimizer.step()
        
        print(f'Train loss: {train_loss/train_norm_variable}')

        torch.save(model.module.state_dict() if isinstance(model, DataParallel) else model.state_dict(), 
                   f'/kaggle/working/model_weights_epoch_{epoch + 3}.pth')
        print(f"Weights saved for epoch {epoch + 3}")
        
        test_loss = 0
        test_norm_variable = 0
        model.eval()
        
        with torch.no_grad():
            for image0, image1, label in test_dataloader:
                image0 = image0.to(device)
                image1 = image1.to(device)
                label = label.to(device)
                
                embed0 = model(image0)
                embed1 = model(image1)
                loss = criterion(embed0, embed1, label)
                summary_loss_test.append(loss.item())
                
                test_loss += loss.item() * image0.size(0)
                test_norm_variable += image0.size(0)
        
        print(f'Test loss: {test_loss/test_norm_variable}')
    return summary_loss_train, summary_loss_test

In [26]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=2e-3)
criterion = nn.CosineEmbeddingLoss()

In [27]:
train(model, optimizer, criterion, 15, train_dataloader, test_dataloader)

Using 2 GPUs


Epochs:   0%|                                                                | 0/15 [00:00<?, ?it/s]

Train loss: 0.1643690459707869
Weights saved for epoch 3


Epochs:   7%|███▍                                                | 1/15 [28:47<6:43:05, 1727.56s/it]

Test loss: 0.13580983809784763
Train loss: 0.12356748970158721
Weights saved for epoch 4


...

Test loss: 0.06451449985935198
Train loss: 0.06713369259714883
Weights saved for epoch 14


Epochs:  80%|███████████████████████████████████████▏         | 12/15 [5:39:00<1:24:29, 1689.70s/it]

Test loss: 0.06293685511386599
Train loss: 0.06582695661458346
Weights saved for epoch 15


Epochs:  80%|███████████████████████████████████████▏         | 12/15 [5:57:53<1:29:28, 1789.48s/it]


KeyboardInterrupt: 